In [1]:
pip install ray ConfigSpace bayesian-optimization

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 152.5 MB/s eta 0:00:0000:0100:01
  Created wheel for ConfigSpace: filename=ConfigSpace-1.2.0-py3-none-any.whl size=115855 sha256=9ff78b52756d72b83be3d712089602500bd74bbe0a15dda1ab1de99315c92818
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5f/1f/4b/450c9c9d1ede888e354984d9eba7e60f13878fa079d502d52e
Successfully built ConfigSpace
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
#import torch
#import torch.optim as optim
#import torch.nn as nn
#from torchvision import datasets, transforms
#from torch.utils.data import DataLoader
#import torch.nn.functional as F
from ray import train, tune # has to be before ray, why?
from ray.tune.schedulers import ASHAScheduler
import bayesopt_search2 as b2

In [3]:
"""This example demonstrates the usage of BayesOpt with Ray Tune.

It also checks that it is usable with a separate scheduler.

Requires the BayesOpt library to be installed (`pip install bayesian-optimization`).
"""
import time
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch

In [52]:
from importlib import reload
reload(b2)

<module 'bayesopt_search2' from '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/bayesopt_search2.py'>

In [53]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        train.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing"
    )
    args, _ = parser.parse_known_args()

#    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0}) # utility function is broken
    # need to manage numerical instability
    algo = b2.BayesOptSearch(bo_args={'batch_size': 50, 'keys': ['width', 'height'], 'n_procs': 4,
                                      'hp_space': {'width':[0, 20], 'height':[-100, 100]}, 
                                      'hp_types': {'height': 'float', 'width': 'float'},
                                      'use_qc': False})
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    tuner = tune.Tuner(
        #easy_objective,
        tune.with_resources(easy_objective, resources={"CPU": 24}),
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            search_alg=algo,
            scheduler=scheduler,
            num_samples=10 if args.smoke_test else 20,
        ),
        run_config=train.RunConfig(
            name="my_exp",
        ),
        param_space={
            "steps": 100,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
        },
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

2024-11-17 03:18:24,156	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ec2-user/ray_results/my_exp' in 0.0170s.
2024-11-17 03:18:24,164	INFO tune.py:1041 -- Total run time: 54.71 seconds (54.68 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'width': 19.148309730364033, 'height': -95.96081184529413}


In [44]:
algo.searcher.test_optimizer.local_buffer

['5.088555004830235,-41.779583444657575',
 '12.007354235354315,-40.48545799666802',
 '12.448950123835871,-35.32304544956513']